In [1]:
import gensim.models as models
from gensim.models import KeyedVectors, Word2Vec 
import numpy as np 
from tqdm import tqdm 
import py_vncorenlp
import gensim

In [2]:

# load wiki word2vec
path_wiki = "./Vecto/wiki.vi.model.bin"
wiki_word2vec_model = KeyedVectors.load_word2vec_format(path_wiki, binary=True)

In [3]:
# load baomoi word2vec
path_baomoi = "./Vecto/baomoi.model.bin"
baomoi_word2vec_model = KeyedVectors.load_word2vec_format(path_baomoi, binary=True)

In [4]:
# load baomoi2 word2vec
path_baomoi2 = "./Vecto/baomoi.window2.vn.model.bin"
baomoi2_word2vec_model = KeyedVectors.load_word2vec_format(path_baomoi2, binary=True)

In [5]:
# load vinAI model 
path_vinAI = "./Vecto/vinAI.bin"
vinAI_model = KeyedVectors.load_word2vec_format(path_vinAI, binary=True)

In [6]:
# baomoi2_word2vec_model.most_similar("du_lịch")
np.array(list(map(list, baomoi2_word2vec_model.most_similar("du_lịch"))))[:,0]

array(['tham_quan', 'du_dịch', 'mice', 'nghỉ_dưỡng', 'thăm_quan',
       'nghỉ_mát', 'lữ_hành', 'ẩm_thực', 'du_lịch_pháp', 'outbound'],
      dtype='<U32')

In [7]:
def savefromtxt(file_name,file_to_save):
    f = open(file_name)
    lines = f.readlines() 
    dict_data = {}
    with open(file_to_save,"w") as s:
        for line in tqdm(lines[3:]):
            s.write(line)
        s.close()
    f.close()

In [8]:
def build_dict_fromtxt(file_name):
    f = open(file_name)
    lines = f.readlines() 
    dict_data = {}
    for line in tqdm(lines[3:]):
        vecto = line.split() 
        key_list = vecto[0:-100]
        key = "_".join(key_list)
        Embedded_vec = np.array(vecto[-100:]).astype(np.float32)
        dict_data[key] = Embedded_vec 

    return dict_data

In [9]:
def save_word2vec_format(fname, vocab, vector_size, binary=True):
    total_vec = len(vocab)
    with open(fname, 'wb') as fout:
        print(total_vec, vector_size)
        fout.write(gensim.utils.to_utf8("%s %s\n" % (total_vec, vector_size)))
        # store in sorted order: most frequent words at the top
        for word, row in tqdm(vocab.items()):
            if binary:
                row = row.astype(np.float32)
                fout.write(gensim.utils.to_utf8(word) + b" " + row.tostring())
            else:
                fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val) for val in row))))
        fout.close()


In [10]:
keys_vinAI_model = vinAI_model.key_to_index 
keys_baomoi = baomoi_word2vec_model.key_to_index 
keys_baomoi2 = baomoi2_word2vec_model.key_to_index 

In [11]:
vinAI_dict = {
    "vocab" : keys_vinAI_model,
    "model" : vinAI_model
}

baomoi_dict = {
    "vocab" : keys_baomoi,
    "model" : vinAI_model
}

baomoi2_dict = {
    "vocab" : keys_baomoi2,
    "model" : vinAI_model
}

dict_model = {
    "vinAI": vinAI_dict,
    "baomoi": baomoi_dict,
    "baomoi2": baomoi2_dict
}

In [12]:
def get_similiar_word(word,dict_model,each_vocab = 20):
    return_list = []
    for key in list(dict_model.keys()):
        vocab = dict_model[key]["vocab"]
        if word in vocab: 
            del vocab 
            model = dict_model[key]["model"]
            this_similars = np.array(list(map(list,model.most_similar(word,topn = each_vocab))))[:,0].tolist()
            return_list.extend(this_similars)
            del model
            del this_similars

    return return_list

In [2]:
# download model 
py_vncorenlp.download_model(save_dir='./vncorenlp')

--2022-07-09 13:54:34--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

     0K .......... .......... .......... .......... ..........  0%  589K 45s
    50K .......... .......... .......... .......... ..........  0%  629K 44s
   100K .......... .......... .......... .......... ..........  0%  792K 40s
   150K .......... .......... .......... .......... ..........  0%  697K 40s
   200K .......... .......... .......... .......... ..........  0% 5.55M 33s
   250K .......... .......... .......... .......... ..........  1%  850K 32s
   300K .......... .......... .......... .......... ..........  1% 1.36M 30s

In [3]:
# Load VnCoreNLP from the local machine folder containing the VnCoreNLP models
model = py_vncorenlp.VnCoreNLP(save_dir='./vncorenlp')

JavaException: JVM exception occurred: vn/pipeline/VnCoreNLP java.lang.NoClassDefFoundError